# visualize_inference_results

### 설명

- 추론 결과 CSV 파일을 기반으로 테스트 이미지를 시각화하는 스크립트입니다.

- CSV 파일에는 모델의 예측 결과가 포함되어 있으며, 각 이미지에 대한 예측 결과를 이미지 위에 표시합니다.

- 시각화된 이미지는 모델 성능을 확인하고 분석하는 데 사용할 수 있습니다.

## Import Library

In [ ]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from tqdm.auto import tqdm

## 경로 설정

- 이미지 디렉토리 및 JSON 파일 경로 설정

In [ ]:
# Test 이미지 또는 validation 이미지 폴더 경로
image_dirs = [
    'path/to/your/test_or_val_folder'
]

# 추론 결과 모델이 예측한 csv 경로
csv_path = 'path/to/your/predict.csv'

### 추론 결과 모델이 예측한 csv, JSON 파일 경로 로드

In [ ]:
try:
    with open(csv_path, 'r') as f:
        data = json.load(f)
except json.JSONDecodeError:
    print('JSON 파일을 파싱할 수 없습니다. 파일 형식을 확인해주세요.')
    data = {}

### 이미지 검색 및 바운딩 박스 시각화 함수 정의

In [ ]:
def find_image_path(image_fname, image_dirs):
    for dir_path in image_dirs:
        potential_path = os.path.join(dir_path, image_fname)
        if os.path.exists(potential_path):
            return potential_path
    return None

In [ ]:
def draw_bboxes_on_image(image, bboxes, color=(0, 255, 0), thickness=2):
    for bbox in bboxes:
        points = bbox
        pts = np.array(points, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image, [pts], isClosed=True, color=color, thickness=thickness)
    return image

## 이미지 시각화

In [ ]:
images_info = data.get('images', {})

pbar = tqdm(total=len(images_info), desc="Processing Images")

# 시각화할 이미지 수 설정
visualize_limit = 5
visualize_count = 0

for image_fname, image_data in images_info.items():
    image_path = find_image_path(image_fname, image_data)

    if image_path is None:
        print(f"이미지를 찾을 수 없습니다. : {image_fname}")
        pbar.updata(1)
        continue

    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 열 수 없습니다. : {image_path}")
        pbar.update(1)
        continue

    words_info = image_data.get('words', {})
    bboxes = [word_info['points'] for word_info in words_info.values()]

    image_with_bboxes = draw_bboxes_on_image(image.copy(), bboxes, color=(0, 255, 0), thickness=2)

    if visualize_count < visualize_limit:
        image_rgb = cv2.cvtColor(image_with_bboxes, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(image_rgb)
        plt.title(f"Image: {image_fname}")
        plt.axis('off')
        plt.show()
        visualize_count += 1
    
    pbar.update(1)

pbar.close()
print(f"모든 이미지를 처리했습니다.")